In [0]:
import pandas as pd
import pandas_profiling
import numpy as np
import ast
import glob
import gc
import re

In [0]:
# from google.colab import files
# uploaded = files.upload()

4/tgHcsUL7tCsUzGzXfi-2X7sN1XpZl_U8xUMjwszmrtgTNqR4dzVAuu8
4/tgGbuVgHZi0DA0rrhSo0KYwHw1usMyg13AaI56N3mPcS9QOEWqr-ao8

In [0]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
link = 'https://drive.google.com/open?id=1811JdzTm2a6lTyDDJc3ejHvLOXuVx_3_' # The shareable link

In [4]:
fluff, id = link.split('=')
print (id) # Verify that you have everything after '='

1811JdzTm2a6lTyDDJc3ejHvLOXuVx_3_


In [5]:
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('Filename.csv')  
df_t = pd.read_csv('Filename.csv')
# Dataset is now stored in a Pandas Dataframe
float_cols = [c for c in df_t if df_t[c].dtype == "float64"]
df = pd.read_csv('Filename.csv',parse_dates=['issue_d'], infer_datetime_format=True, engine='c', dtype={c: np.float32 for c in float_cols})

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (18,46,54) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
del df_t
gc.collect()

4

In [7]:
link = 'https://drive.google.com/open?id=1JEIwYsLXeuONQ3qypd9SzRUe2fJDQg48' # The shareable link
fluff, id = link.split('=')
print (id) # Verify that you have everything after '='

1JEIwYsLXeuONQ3qypd9SzRUe2fJDQg48


In [8]:
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('Filename.xlsx')  
browse_notes = pd.read_excel('Filename.xlsx',
                             sheet_name=1)
browse_notes.head()

,BrowseNotesFile,Description,Exist in LoanStats
0,acceptD,The date which the borrower accepted the offer,NaN
1,accNowDelinq,The number of accounts on which the borrower i...,NaN
2,accOpenPast24Mths,Number of trades opened in past 24 months.,NaN
3,addrState,The state provided by the borrower in the loan...,NaN
4,all_util,Balance to credit limit on all trades,all_util


In [9]:
browseNotes_v = browse_notes['BrowseNotesFile'].dropna().values
browseNotes_v = [re.sub('(?<![0-9_])(?=[A-Z0-9])', '_', x).lower().strip() for x in browseNotes_v]
LoanStats_v = df.columns.values
IncorrectColumns = ['is_inc_v', 'mths_since_most_recent_inq', 'mths_since_oldest_il_open',
         'mths_since_recent_loan_delinq', 'verified_status_joint']
MissingColumns = ['verification_status', 'mths_since_recent_inq', 'mo_sin_old_il_acct',
           'mths_since_recent_bc_dlq', 'verification_status_joint']

browse_feat = np.setdiff1d(browseNotes_v, IncorrectColumns)
browse_feat = np.append(browseNotes_v, MissingColumns)
avail_feat = np.intersect1d(browseNotes_v, LoanStats_v)
X = df[avail_feat].copy()
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1321847 entries, 0 to 1321846
Data columns (total 86 columns):
acc_now_delinq                    1321818 non-null float32
acc_open_past_24mths              1271817 non-null float32
addr_state                        1321847 non-null object
all_util                          455695 non-null float32
annual_inc                        1321843 non-null float32
annual_inc_joint                  9300 non-null float32
application_type                  1321847 non-null object
avg_cur_bal                       1251559 non-null float32
bc_open_to_buy                    1259244 non-null float32
bc_util                           1258527 non-null float32
chargeoff_within_12_mths          1321702 non-null float32
collections_12_mths_ex_med        1321702 non-null float32
delinq_2yrs                       1321818 non-null float32
delinq_amnt                       1321818 non-null float32
desc                              126067 non-null object
dti       

In [10]:
eda_report = pandas_profiling.ProfileReport(X)

/usr/local/lib/python3.6/dist-packages/pandas_profiling/describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


In [11]:
eda_report

Number of variables,86
Number of observations,1321847
Total Missing (%),18.4%
Total size in memory,509.3 MiB
Average record size in memory,404.0 B
Numeric,65
Categorical,14
Boolean,0
Date,0
Text (Unique),0
Rejected,7


In [0]:
eda_report.to_file("LC_EDA_v1.html")

In [0]:
rejected_variables = eda_report.get_rejected_variables(threshold=0.9)

In [0]:
rejected_variables

['installment',
 'loan_amnt',
 'member_id',
 'num_rev_tl_bal_gt_0',
 'open_acc',
 'tot_hi_cred_lim',
 'total_il_high_credit_limit']

In [0]:
from google.colab import files
files.download('LC_EDA_v1.html')

MessageError: ignored

In [0]:
pandas_profiling.ProfileReport(X)

In [0]:
pandas_profiling()